# Dados por partida

In [18]:
#importa os pacotes necessarios
from requests import status_codes, get
from bs4 import BeautifulSoup
import datetime
import pandas as pd

Tendo como base os dados disponibilizados pela CBF, podemos escolher um ano e um jogo da temporada (como o quarto jogo, por exemplo). Com isso, seremos capazes de pegar todos os links associados aos jogos aquele ano.

In [19]:
# EXEMPLO

# Série do campeonato brasileiro
serie = 'A'
# Temporada
temporada = 2018
# Link com a série Definida
link = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie' + f'-{serie.lower()}/'
#Link com o ano definido
link += str(temporada)+'/'
# Jogo (Quarto jogo da temporada 2018)
jogo = 4
# Link do quarto jogo da temporada
link += str(jogo)

# Printa o link do jogo
print(f'O Link do jogo é: {link}')

# Valida o link
def validador(url):
    if get(url).status_code == 200:
        return 'está retornando informações do site'
    else:
        return 'não está retornando informações do site'

# Testa se está funcionando corretamente
print(f'O link {validador(link)}')

# OBS: Se você mudar o valor do ano ou do jogo par aalgum valor absurdo, o validador retornar que as informações não são retornadas.
# Experimente mudar esses valores.

O Link do jogo é: https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a/2018/4
O link está retornando informações do site


### Pegando os dados da temporada vigente.


In [20]:
# Temporada do ano vigente.
temporada = datetime.datetime.now().year

# Série A
link_a = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a/'
link_a += str(temporada)+'/'

# Série B
link_b = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-b/'
link_b += str(temporada)+'/'

# Série C
link_c = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-c/'
link_c += str(temporada)+'/'

print(f'O link da Série A {validador(link_a)}')
print(f'O link da Série B {validador(link_b)}')
print(f'O link da Série C {validador(link_c)}')

O link da Série A está retornando informações do site
O link da Série B está retornando informações do site
O link da Série C está retornando informações do site


In [21]:
#Cria metodo para buscar e salvas os dados requiridos
def busca_dado_partida(link_partida):
  # Request para o link da partida
  req = get(link_partida)
  # pegando os dados em HTML
  soup = BeautifulSoup(req.text, 'html.parser')
  # Dados da partida
  serie = soup.find(class_ = 'color-white block').text.split('-')[1].strip().split(' ')[1]
  temporada = int(soup.find(class_ = 'color-white block').text.split('-')[2].strip()) 
  estadio = soup.find_all(class_ = 'text-2 p-r-20')[0].text.split('-')[0].strip() 
  cidade = soup.find_all(class_ = 'text-2 p-r-20')[0].text.split('-')[1].strip()
  estado = soup.find_all(class_ = 'text-2 p-r-20')[0].text.split('-')[2].strip()
  dia_semana = soup.find_all(class_ = 'text-2 p-r-20')[1].text.split(',')[0].strip()
  dia_numero = int(soup.find_all(class_ = 'text-2 p-r-20')[1].text.split(',')[1].strip().split(' ')[0])
  mes = soup.find_all(class_ = 'text-2 p-r-20')[1].text.split(',')[1].strip().split('de')[1]
  ano = int(soup.find_all(class_ = 'text-2 p-r-20')[1].text.split(',')[1].strip().split('de')[2]) 
  hora = soup.find_all(class_ = 'text-2 p-r-20')[2].text.strip()
  mandante = soup.find_all(class_ = 'time-nome color-white')[0].text.strip()
  visitante = soup.find_all(class_ = 'time-nome color-white')[1].text.strip() 
  gol_mandante = int(soup.find_all(class_ = 'time-gols block')[0].text.strip())
  gol_visitante = int(soup.find_all(class_ = 'time-gols block')[1].text.strip())

  if gol_mandante > gol_visitante:
    vitoria_mandante = 1
    vitoria_visitante = 0
    jogo_empatado = 0
    ponto_mandante = 3
    ponto_visitante = 0
  if gol_mandante < gol_visitante:
    vitoria_mandante = 0
    vitoria_visitante = 1
    jogo_empatado = 0
    ponto_mandante = 0
    ponto_visitante = 3
  if gol_mandante == gol_visitante:
    vitoria_mandante = 0
    vitoria_visitante = 0
    jogo_empatado = 1
    ponto_mandante = 1
    ponto_visitante = 1


  # Cria um dicionario para estruturar os dados
  partida = {
      'serie' : serie,
      'temporada' : temporada,
      'estadio' : estadio,
      'cidade' : cidade,
      'estado' : estado,
      'dia_semana' : dia_semana,
      'dia_numero' : dia_numero,
      'mes' : mes,
      'ano' : ano,
      'hora' : hora,
      'mandante' : mandante,
      'visitante' : visitante,
      'gol_mandante' : gol_mandante,
      'gol_visitante' : gol_visitante,
      'vitoria_mandante' : vitoria_mandante,
      'vitoria_visitante' : vitoria_visitante,
      'jogo_empatado' : jogo_empatado,
      'ponto_mandante' : ponto_mandante,
      'ponto_visitante' : ponto_visitante
  }
  # Código obtido de Marcelo Alexandre - 2021 - linkedin.com/in/marcelocristianoalexandre/
  return(partida)

# Testando a função
print(busca_dado_partida('https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a/2018/4'))

{'serie': 'A', 'temporada': 2018, 'estadio': 'Manoel Barradas', 'cidade': 'Salvador', 'estado': 'BA', 'dia_semana': 'Sábado', 'dia_numero': 14, 'mes': ' Abril ', 'ano': 2018, 'hora': '19:00', 'mandante': 'Vitória - BA', 'visitante': 'Flamengo - RJ', 'gol_mandante': 2, 'gol_visitante': 2, 'vitoria_mandante': 0, 'vitoria_visitante': 0, 'jogo_empatado': 1, 'ponto_mandante': 1, 'ponto_visitante': 1}


Atualmente, cada temporada decorre entre maio e dezembro, tendo 38 rodadas com dez partidas cada, totalizando 380 partidas em toda a temporada.

In [22]:
array_data_a = []
array_data_b = []
array_data_c = []

# Ranfe deve ser de range(1, 381)
for i in range(1, 382):
    url_a = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a/'
    url_b = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-b/'
    url_c = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-c/'
    url_a += f'{temporada}/{i}'
    url_b += f'{temporada}/{i}'
    url_c += f'{temporada}/{i}'
    try:
        array_data_a.append(busca_dado_partida(url_a))
        array_data_b.append(busca_dado_partida(url_b))
        array_data_c.append(busca_dado_partida(url_c))
        print(f'Dado Da partida {i} adicionado')
    except AttributeError:
        print(f'Dado Da partida {i} NÂO adicionado')


DadosPartidasA = pd.DataFrame(columns=array_data_a[0].keys(), data=[data.values() for data in array_data_a])
DadosPartidasB = pd.DataFrame(columns=array_data_b[0].keys(), data=[data.values() for data in array_data_b])
DadosPartidasC = pd.DataFrame(columns=array_data_c[0].keys(), data=[data.values() for data in array_data_c])
#print(DadosPartidasA)
#print(DadosPartidasB)
#print(DadosPartidasC)

DadosPartidasA.to_csv('Serie_A_{temporada}.csv')
DadosPartidasB.to_csv('Serie_B_{temporada}.csv')
DadosPartidasC.to_csv('Serie_C_{temporada}.csv')

Dado Da partida 1 adicionado
Dado Da partida 2 adicionado
Dado Da partida 3 adicionado
Dado Da partida 4 adicionado
Dado Da partida 5 adicionado
Dado Da partida 6 adicionado
Dado Da partida 7 adicionado
Dado Da partida 8 adicionado
Dado Da partida 9 adicionado
Dado Da partida 10 adicionado
Dado Da partida 11 adicionado
Dado Da partida 12 adicionado
Dado Da partida 13 adicionado
Dado Da partida 14 adicionado
Dado Da partida 15 adicionado
Dado Da partida 16 adicionado
Dado Da partida 17 adicionado
Dado Da partida 18 adicionado
Dado Da partida 19 adicionado
Dado Da partida 20 adicionado
Dado Da partida 21 adicionado
Dado Da partida 22 adicionado
Dado Da partida 23 adicionado
Dado Da partida 24 adicionado
Dado Da partida 25 adicionado
Dado Da partida 26 adicionado
Dado Da partida 27 adicionado
Dado Da partida 28 adicionado
Dado Da partida 29 adicionado
Dado Da partida 30 adicionado
Dado Da partida 31 adicionado
Dado Da partida 32 adicionado
Dado Da partida 33 adicionado
Dado Da partida 34 

# Hands-On: Dados Por Partida

* Pegar Dados das Escalações
* Pegar Dados da Arbitragem
___

* Montar Banco de Dados
